In [1]:
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Topology import Topology
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary
from topologicpy.Plotly import Plotly
from topologicpy.Neo4j import Neo4j

In [2]:
p = CellComplex.Prism(width=2, uSides=5, vSides=3, wSides=4, placement="bottom")
c = Cell.Prism(width=0.8, length=0.8, height=1.2, placement="bottom")
p = Topology.Impose(p, c)
c = Cell.Prism(width=0.8, length=1.4, height=1.2, placement="bottom")
p = Topology.Merge(p, c)

print(p)
# Topology.Show(p, renderer="notebook")

In [3]:
# cluster = Topology.Explode(p, scale=1.5)
# Topology.Show(cluster, faceOpacity=1)

In [4]:
brep = Topology.BREPString(p)
# print(brep)

In [5]:
d = CellComplex.Decompose(p)
d['Cells'] = Topology.Cells(p)
d['brep'] = brep
keys = Dictionary.Keys(d)
vertexGroups = []
for key in keys:
    topologies = d[key]
    if len(topologies) > 0:
        vertexGroups.append(key)
    # print(topologies)
    # print(keys)
# print(vertexGroups)
       
def assignDictionary(topologies, vertexGroup):
    for i, t in enumerate(topologies):
        d = Dictionary.ByKeysValues(["id","group", "brep"], [vertexGroup+str(i),vertexGroup, brep])
        t = Topology.SetDictionary(t, d)
        # print(t)
        v = Dictionary.ValueAtKey(d, "group")
        # print(v)
    
for vertexGroup in vertexGroups:
    topologies = d[vertexGroup]
    assignDictionary(topologies, vertexGroup)
    # print(topologies)
    print(vertexGroup, (type(topologies)))
    # print(vertexGroups)

externalVerticalFaces <class 'list'>
internalVerticalFaces <class 'list'>
topHorizontalFaces <class 'list'>
bottomHorizontalFaces <class 'list'>
internalHorizontalFaces <class 'list'>
Cells <class 'list'>
brep <class 'str'>


In [6]:
g = Graph.ByTopology(p, toExteriorTopologies=True, viaSharedTopologies=True, storeBRep=False)
data01 = Plotly.DataByGraph(g, vertexGroupKey="group", vertexLabelKey="group",
                            vertexGroups=vertexGroups, colorScale="thermal")
data02 = Plotly.DataByTopology(p, faceOpacity=0)
fig = Plotly.FigureByData(data01+data02)
# Plotly.Show(fig)

In [7]:
Graph.PyvisGraph(g, "C:/Users/anyde/Documents/PythonTutorials/TopologicPy/pyvisTutorial/Diagram.html",
                 backgroundColor="white",
                 fontColor="black", vertexLabelKey="group", vertexColor="#456789",
                 vertexGroupKey="group", vertexGroups=vertexGroups,
                 vertexSize=0, vertexSizeKey="degree", vertexColorKey=None,
                 edgeWeightKey=None, notebook=False, colorScale="thermal")

C:/Users/anyde/Documents/PythonTutorials/TopologicPy/pyvisTutorial/diagram.html


In [8]:
# Connect to Neo4j database
neoGraph = Neo4j.ByParameters("bolt://localhost:7687", "neo4j", "00000000")

# Add Topologic graph to Neo4j database
Neo4j.SetGraph(neoGraph, g, labelKey='group')

Graph('bolt://localhost:7687')